In [8]:
import pandas as pd
import os 
import matplotlib.pyplot as plt 
import re
import numpy as np

In [9]:
# load tree data from the municipality of Copenhagen 
cols = ['id','ny_dm_element','slaegtsnavn','planteaar','bydelsnavn','vejnavn','placering','wkb_geometry']
df = pd.read_csv('raw_data/trae_basis.csv',sep=',',usecols=cols)

# prepare tree data for analysis
df = df[df["planteaar"] >= 2000] # get data from 2000 and after 
df = df.dropna(subset=['planteaar']) # remove samples with unknown plant year 
df.columns = ['id','type','species','year','district','street','street_location','coordinates']

# Get the X and Y coordinates for geodata plot
def get_x(row):
    x = re.findall(r"[-+]?\d*\.\d+|\d+", row)[0]
    return float(x)
def get_y(row):
    y = re.findall(r"[-+]?\d*\.\d+|\d+", row)[1]
    return float(y)

df['X'] = df['coordinates'].apply(get_x)
df['Y'] = df['coordinates'].apply(get_y)
df = df.drop(columns = ["coordinates"])

# save tree data for exploratory tree 
df.year = df.year.apply(int)
df = df[df['year']<2022]
df.to_csv('data/df_tree.csv', index=False) 

In [20]:
# load income data from the municipality of Copenhagen 
df1 = pd.read_excel('raw_data/2022419113038370864435KKIND3.xlsx', header=2, usecols = "C:Y")
df1.rename(columns = {'Unnamed: 2':'income_type', 'Unnamed: 3':'district'}, inplace = True)
df1['income_type'] = df1['income_type'].fillna(method='ffill')
df1['district'] = df1['district'].str.replace("Bydel - ", "")
df1['district'] = df1['district'].str.replace("/", "-")
df1 = df1[df1['district']!='København i alt']

df_income = df1.melt(id_vars=["district","income_type"],var_name="year", value_name="total")
df_income = (pd.pivot_table(df_income, values = "total", index = ["year", "district"], columns = ["income_type"], aggfunc = "sum")).reset_index()
df_income = df_income.drop(columns=['Indkomstbeløb (1.000 kr.)','Personer med indkomsten (antal)'])
df_income.rename(columns={'Gennemsnit for alle personer med indkomsten (kr.)': 'avg_income'}, inplace=True)
df_income['year'] = df_income['year'].astype(int)
df_income.to_csv('data/df_income.csv', index=False) # save income data

In [21]:
# load education data 
df2 = pd.read_excel('raw_data/2022419114031370864435KKUDD2.xlsx', header =2, usecols = "C:R")
df2.rename(columns = {'Unnamed: 2':'race', 'Unnamed: 3':'education', 'Unnamed: 4':'district'}, inplace = True)
df2['race'] = df2['race'].fillna(method='ffill')
df2['education'] = df2['education'].fillna(method='ffill')
df2['district'] = df2['district'].str.replace("Bydel - ", "")
df2['district'] = df2['district'].str.replace("/", "-")
df2 = df2[df2['district']!='København i alt']

df_education = df2.melt(id_vars=["district","race", "education"], var_name="year", value_name="total_educated")
df_education['year'] = df_education['year'].apply(int)
df_race = (pd.pivot_table(df_education, values = "total_educated", index = ["year", "district"], columns = ["race"], aggfunc = "sum")).reset_index()
df_level = (pd.pivot_table(df_education, values = "total_educated", index = ["year", "district"], columns = ["education"], aggfunc = "sum")).reset_index()

df_merged = pd.merge(df_race,df_level)

# Get average number of years in education per district
grundskole = 10
erhvervsfaglig = grundskole + 2
gymnasiel = grundskole + 3
lang_videregående = gymnasiel + (5+8)/2
mellem_videregående = gymnasiel + 3

# calculate average number of study year per district
df_merged['total_study_year'] = (df_merged['Erhvervsfaglige og korte videregående uddannelser']*erhvervsfaglig
+ df_merged['Grundskole og uoplyst mv.']*grundskole
+ df_merged['Gymnasiale uddannelser og adgangsgivende uddannelsesforløb']*gymnasiel
+ df_merged['Lange videregående uddannelser og Ph.d og forskeruddannelser']*lang_videregående
+ df_merged['Mellemlange videregående uddannelser og bacheloruddannelser']*mellem_videregående)
df_merged['total_educated'] = df_merged[['Erhvervsfaglige og korte videregående uddannelser','Grundskole og uoplyst mv.','Gymnasiale uddannelser og adgangsgivende uddannelsesforløb','Lange videregående uddannelser og Ph.d og forskeruddannelser','Mellemlange videregående uddannelser og bacheloruddannelser']].sum(axis=1)
df_merged['avg_study_year'] = df_merged.total_study_year/df_merged.total_educated

df_merged.to_csv('data/df_education.csv', index=False) # save education data 

In [22]:
# load area size data 
df_size = pd.read_excel('raw_data/2022426114414371694040KKAREAL.xlsx', header=2, usecols = "B:K")
df_size = (df_size.dropna()).transpose()
df_size = df_size.reset_index()
df_size.rename(columns = {"index":'district', 0:'area_m2'}, inplace = True)
df_size['district'] = df_size['district'].str.replace("Bydel - ", "")
df_size['district'] = df_size['district'].str.replace("/", "-")
df_size['district'] = df_size['district'].str.replace("Kgs", "Kongens")
df_size['area_km2'] = df_size['area_m2']/(1000*1000)
df_size.to_csv('data/df_size.csv', index=False) # save area size data 

In [23]:
## Merge datasets

# Get number of planted trees and number of unique species per year per district 
data_tree = (df.groupby(['year','district']).size()).reset_index()
df_species = df[['year','district','species']].groupby(['year','district']).nunique()
df_species =  df_species.reset_index()
df_tree_merged = pd.merge(data_tree,df_species)

# Get number of planted tree types per year per district 
df_type = df[['year','district','type','id']].groupby(['year','district','type']).count()
data_type = (df_type.pivot_table(index=['year','district'],columns='type', values='id')).reset_index()
data_type = data_type.fillna(0)
data_type = data_type.drop(columns='0')

# Merge above dataframes 
df_tree_merged = pd.merge(df_tree_merged, data_type)
df_tree_merged.rename(columns={0: 'number_planted_trees'}, inplace=True)

# load area size data
df_tree_final = pd.merge(df_tree_merged,df_size[['district','area_km2']])
df_tree_final['trees_km2'] = df_tree_final.number_planted_trees/df_tree_final.area_km2 

df_genial = pd.merge(pd.merge(df_tree_final,df_income),df_merged[['year','district','avg_study_year']], how='outer')
df_genial.to_csv('data/df_genial.csv', index=False)